# Analysis

This notebook contains the code necessary to conduct all data analyses for the Ancient Genotyped Variants Proxy Catalog project. I have organized sections by order of appearance in the manuscript. Use the Table of Contents below to navigate to specific analyses.

## Table of Contents

- [Notebook Setup](#notebooksetup)
- [Descriptive Stats](#descriptivestats)
- [N AGV-LDV Pairs](#nagvldvpairs)
- [Proportion TopLD Variants Among MAF Class](#proportiontopldvariantsamongmafclasses)
- [N and Proportion TopLD Variants in LD with AGVs at Varying LD Thresholds](#nandproportiontopldvariantsinldwithagvsatvaryingldthresholds)
- [AGV and LD Variants GWAS Intersection](#gwasintersection)
- [Evaluation](#evaluation)
    - [Mismatch Proportions by LD Threshold](#mismatchesbythreshold)
    - [Mismatches by Genomic Window](#mismatchesbywindow)
- [AGVs in TopLD](#agvsintopld)
- [AGV Age Estimates](#agvageestimates)
- [Selection GWAS Loci Allele Frequency Trajectories](#gwastrajectories)

## Notebook Setup <a class = 'anchor' id = 'notebooksetup'></a>

Let's start with setup and import the needed packages.

In [1]:
import glob
import gzip
import numpy as np
import os
import pandas as pd
import pybedtools
import subprocess
from collections import Counter, OrderedDict
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr

pd.options.display.max_columns = 30
pd.options.display.max_rows = 30

## Descriptive Stats <a class = 'anchor' id = 'descriptivestats'></a>

How many total unique TopLD variants are there?

In [2]:
!cat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/scripts/TopLD/calculate_N_unique_TopLD_variants.out

Total unique variants: 223322434


How many hg19 AGVs are present in v54.1 of the AADR?

In [3]:
!wc -l /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg19.bed | awk '{print $1}'

1233013


How many samples and unique ancient individuals (Date mean > 0) are represented by v54.1 of the AADR?

In [4]:
AADR_annotation = pd.read_csv('/wynton/group/capra/data/wynton_databases/ancient_dna/V54.1.p1/v54.1.p1_1240K_public.anno', sep='\t', header=0)

/scratch/cbrand/ipykernel_3155226/1007409817.py:1: DtypeWarning: Columns (14,15,18,26,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  AADR_annotation = pd.read_csv('/wynton/group/capra/data/wynton_databases/ancient_dna/V54.1.p1/v54.1.p1_1240K_public.anno', sep='\t', header=0)


In [5]:
print(len(AADR_annotation[AADR_annotation['Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]'] > 0]), "samples")

9989 samples


In [6]:
print(len(AADR_annotation[AADR_annotation['Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]'] > 0]['Master ID'].unique()), "unique individuals")

9623 unique individuals


How many AGVs could be lifted over to hg38?

In [7]:
!wc -l /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg38.bed | awk '{print $1}'

1232486


How many were not successful?

In [8]:
output = !wc -l /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg19.unlifted
lines = int(output[0].split()[0])
result = lines // 2 # divide by 2 because each unlifted variant has a comment line
print(result)

527


How are the failed sites distributed across chromosomes?

In [9]:
chrs = []
with open('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg19.unlifted','r') as unlifted:
    for line in unlifted:
        if not line.startswith('#'):
            chrs.append(line.split('\t')[0])
            
chr_distribution = Counter(chrs)
for chr, count in chr_distribution.items():
    print(f'{chr}: {count}')

chr1: 47
chr2: 16
chr3: 19
chr4: 1
chr6: 16
chr7: 21
chr8: 43
chr9: 15
chr10: 4
chr11: 8
chr12: 2
chr13: 5
chr14: 52
chr15: 1
chr17: 35
chr18: 1
chr19: 43
chr20: 4
chr21: 6
chr22: 53
chrX: 123
chrY: 12


How many successfully lifted over variants do not occur on chromosome Y because we do not have LD information for that chromosome?

In [10]:
!grep -c -v 'chrY' /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg38.bed

1199828


How common are AGVs in complex genomic regions (e.g., segmental duplications and structural variants)?

In [11]:
AGVs_hg38_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg38.bed')
AGVs_hg38_pbtBED.head(5)

chr1	817185	817186	G	A	rs3094315
 chr1	841165	841166	A	G	rs12124819
 chr1	897537	897538	T	C	rs28765502
 chr1	906632	906633	T	G	rs7419119
 chr1	911483	911484	G	C	rs950122
 

In [12]:
SDs_hg38_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/annotations/SDs_hg38.bed')
SDs_hg38_pbtBED.head(5)

chr1	10000	19844
 chr1	10000	40733
 chr1	10001	37148
 chr1	10464	19844
 chr1	10464	40733
 

In [13]:
SVs_hg38_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/annotations/HGSVC_2024_v1_SVs_hg38.bed')
SVs_hg38_pbtBED.head(5)

chr1	10669	10901
 chr1	10774	10775
 chr1	10786	10787
 chr1	10797	10798
 chr1	10828	10829
 

In [14]:
SDs_AGVs_intersect = SDs_hg38_pbtBED.intersect(AGVs_hg38_pbtBED)
len(SDs_AGVs_intersect)

25718

In [15]:
SVs_AGVs_intersect = SVs_hg38_pbtBED.intersect(AGVs_hg38_pbtBED)
len(SVs_AGVs_intersect)

13270

In [16]:
ancient_sample_coverage = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/ancient_sample_coverage_per_site/ancient_sample_coverage_per_site.txt', sep='\t', header=0)
ancient_sample_coverage.head(5)

,chr,pos,non_missing_samples
0,chr1,752566,4535
1,chr1,776546,4985
2,chr1,832918,2650
3,chr1,842013,4520
4,chr1,846864,4899


In [17]:
ancient_sample_coverage['non_missing_samples'].median()

np.float64(4550.0)

In [18]:
ancient_sample_coverage['non_missing_samples'].min()

np.int64(0)

In [19]:
len(ancient_sample_coverage[ancient_sample_coverage['non_missing_samples'] == 0])

18

In [20]:
ancient_sample_coverage[ancient_sample_coverage['non_missing_samples'] == 0]

,chr,pos,non_missing_samples
49745,chr1,149810915,0
93113,chr1,248612910,0
139998,chr2,112037028,0
147566,chr2,131315032,0
372436,chr5,70225669,0
372440,chr5,70324611,0
439255,chr6,31950247,0
524919,chr7,62878463,0
528110,chr7,74269928,0
537161,chr7,100641872,0


In [21]:
ancient_sample_coverage['non_missing_samples'].max()

np.int64(8124)

## N AGV-LDV Pairs <a class = 'anchor' id = 'nagvldvpairs'></a>

How many total AGV-LDV pairs exist?

In [22]:
#directory = '/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants/'
#file_suffix = '_AGV_LDVs_summary.txt.gz'

#N_variant_pairs_per_chr = OrderedDict()
#chrs = [str(i) for i in range(1, 23)] + ['X']

#output_file_path = os.path.join(directory, 'N_AGV_LD_variant_pairs_by_chromosome.txt')
#with open(output_file_path, 'w') as output_file:

#   for chr in chrs:
#       file_name = f'chr{chr}{file_suffix}'
#       file_path = os.path.join(directory, file_name)

#       if os.path.exists(file_path):
#           with gzip.open(file_path, 'rt') as file:
#               lines = file.readlines()[1:]  # Skip header
#               chr = file_name.split('_')[0]
#               N_variant_pairs_per_chr[chr] = len(lines)

#   for chr, N_variant_pairs in N_variant_pairs_per_chr.items():
#       output_file.write(f'{chr}: {N_variant_pairs}\n')

#   total_rows = sum(N_variant_pairs_per_chr.values())
#   output_file.write(f'N Total Unique AGV-LDV pairs: {total_rows}\n')

#print(f'Output saved to: {output_file_path}')

In [23]:
!cat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants/N_AGV_LD_variant_pairs_by_chromosome.txt

chr1: 18853532
chr2: 21570407
chr3: 18862705
chr4: 17533078
chr5: 16943569
chr6: 22676664
chr7: 13490604
chr8: 14943881
chr9: 10359368
chr10: 13108861
chr11: 12969412
chr12: 11753683
chr13: 8400589
chr14: 7709272
chr15: 6574979
chr16: 6230628
chr17: 5270539
chr18: 6341375
chr19: 3501808
chr20: 4864793
chr21: 2734315
chr22: 2557739
chrX: 13480874
N Total Unique AGV-LDV pairs: 260732675


How many AGV-LDV pairs occur in each ancestry group?

In [24]:
#directory = '/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants/'
#file_suffix = '_AGV_LDVs_summary.txt.gz'

#chrs = [str(i) for i in range(1, 23)] + ['X']
#ancestry_groups = ['AFR', 'EAS', 'EUR', 'SAS']
#count_dict = {key: 0 for key in ancestry_groups}

#output_file_path = os.path.join(directory, 'N_AGV_LD_variant_pairs_by_ancestry_group.txt')

#for ancestry_group in ancestry_groups:
#    total = 0
#    for chr in chrs:
#        file_name = f'chr{chr}{file_suffix}'
#       file_path = os.path.join(directory, file_name)

#        if os.path.exists(file_path):
#            result = subprocess.run(['zgrep', '-c', ancestry_group, file_path], stdout=subprocess.PIPE, stderr=subprocess.DEVNULL, text=True)
#            count = int(result.stdout.strip()) if result.stdout.strip().isdigit() else 0
#            total += count
#    count_dict[ancestry_group] = total

#with open(output_file_path, 'w') as f:
#    for ancestry_group, count in count_dict.items():
#        f.write(f'Total occurrences of {ancestry_group}: {count}\n')

#print(f'Output saved to: {output_file_path}')

In [25]:
!cat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants/N_AGV_LD_variant_pairs_by_ancestry_group.txt

Total occurrences of AFR: 87797014
Total occurrences of EAS: 138035897
Total occurrences of EUR: 156025349
Total occurrences of SAS: 128135879


## Proportion TopLD Variants Among MAF Classes <a class = 'anchor' id = 'proportiontopldvariantsamongmafclasses'></a>

What is the distribution of variant MAF classes among the four ancestry groups in TopLD?

In [26]:
proportion_TopLD_variants_per_ancestry_group_per_MAF_class = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/TopLD/proportion_TopLD_variants_per_ancestry_group_per_MAF_class.txt', sep='\t', header=0)
proportion_TopLD_variants_per_ancestry_group_per_MAF_class

,ancestry_group,MAF_class,proportion
0,AFR,UR,0.512049
1,AFR,R,0.230152
2,AFR,LF,0.113081
3,AFR,C,0.144717
4,EAS,UR,0.530110
5,EAS,R,0.248446
6,EAS,C,0.159941
7,EAS,LF,0.061503
8,EUR,UR,0.897654
9,EUR,R,0.043853


## N and Proportion TopLD Variants in LD with AGVs at Varying LD Thresholds <a class = 'anchor' id = 'nandproportiontopldvariantsinldwithagvsatvaryingldthresholds'></a>

What are the counts and proportions of TopLD variants in LD at a given threshold with at least one AGV among the four ancestry groups at varying LD thresholds?

In [27]:
def concat_N_TopLD_variants_in_LD_with_AGVs_files():
    """
    Reads multiple text files matching a pattern, extracts a variable from the filename,
    adds a new column with the variable, and concatenates all files into one DataFrame.

    Parameters:
    - file_pattern (str): Glob pattern to match filenames (e.g., "data_*.txt").
    - variable_position (int): The position of the variable in the filename when split by an underscore.
    - delimiter (str): Delimiter used in the text files (default is tab-delimited).

    Returns:
    - pd.DataFrame: Concatenated DataFrame with an additional column for the variable.
    """
    data = []
    ancestry_groups = ['AFR','EAS','EUR','SAS']

    for ancestry_group in ancestry_groups:
        file_path = os.path.join('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants', f'{ancestry_group}_TopLD_variants_in_LD_with_AGVs.txt')
        for file in glob.glob(file_path):
            df = pd.read_csv(file, sep='\t', header=0)
            df['ancestry_group'] = ancestry_group
            df = df[['ancestry_group','MAF_class','R2_0.2','R2_0.5','R2_0.6','R2_0.7','R2_0.8','R2_0.9','R2_0.92','R2_0.94','R2_0.96','R2_0.98','R2_1','Sum']]            
            data.append(df)

    concatenated_df = pd.concat(data, ignore_index=True)

    return concatenated_df

In [28]:
N_TopLD_variants_in_LD_with_AGVs = concat_N_TopLD_variants_in_LD_with_AGVs_files()
N_TopLD_variants_in_LD_with_AGVs

,ancestry_group,MAF_class,R2_0.2,R2_0.5,R2_0.6,R2_0.7,R2_0.8,R2_0.9,R2_0.92,R2_0.94,R2_0.96,R2_0.98,R2_1,Sum
0,AFR,UR,1135209,643837,372274,349570,349570,349570,349570,349570,349570,349570,349570,32165070
1,AFR,R,1549944,628429,532138,435018,356358,277326,260176,247282,231699,228122,228122,14457324
2,AFR,LF,3102526,1290247,1048892,863875,698598,522219,479430,428070,368150,281545,206709,7103365
3,AFR,C,8033882,5985626,5357188,4710670,3991442,3116337,2895375,2640908,2325497,1851755,567216,9090580
4,EAS,UR,1308976,1027423,779722,779722,779722,779722,779722,779722,779722,779722,779722,19553587
5,EAS,R,1164863,829613,695518,625110,581846,526434,519890,507644,506172,506172,506172,9164135
6,EAS,LF,1214328,967195,913886,856989,788120,680099,644810,604087,537512,452932,405850,2268582
7,EAS,C,5622288,5160208,4970029,4746858,4450699,3952501,3788008,3570656,3252480,2673157,1154193,5899580
8,EUR,UR,5367310,2992388,2026523,1699717,1501551,1293541,1255617,1229263,1204432,1199025,1199025,140512443
9,EUR,R,1343295,934561,836898,734178,617535,467551,427916,378168,324162,240885,163188,6864507


Let's save this dataframe for the supplement.

In [29]:
N_TopLD_variants_in_LD_with_AGVs.to_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/supplemental_dfs/N_TopLD_variants_in_LD_with_AGVs.txt', sep='\t', header=True, index=False)

In [30]:
def proportion_TopLD_variants_in_LD_with_AGVs_files(df):
    """
    Calculates proportions for count columns based on the 'Sum' column in the raw counts DataFrame.

    Parameters:
    - raw_counts_df (pd.DataFrame): DataFrame with raw counts.

    Returns:
    - pd.DataFrame: DataFrame with proportions calculated for each count column.
    """
    count_cols = df.columns[2:13]
    df[count_cols] = df[count_cols].div(df['Sum'], axis=0)
    proportions_df = df.drop(['Sum'], axis=1)

    return proportions_df

In [31]:
proportion_TopLD_variants_in_LD_with_AGVs = proportion_TopLD_variants_in_LD_with_AGVs_files(N_TopLD_variants_in_LD_with_AGVs)
proportion_TopLD_variants_in_LD_with_AGVs

,ancestry_group,MAF_class,R2_0.2,R2_0.5,R2_0.6,R2_0.7,R2_0.8,R2_0.9,R2_0.92,R2_0.94,R2_0.96,R2_0.98,R2_1
0,AFR,UR,0.035293,0.020017,0.011574,0.010868,0.010868,0.010868,0.010868,0.010868,0.010868,0.010868,0.010868
1,AFR,R,0.107208,0.043468,0.036808,0.030090,0.024649,0.019182,0.017996,0.017104,0.016026,0.015779,0.015779
2,AFR,LF,0.436768,0.181639,0.147661,0.121615,0.098347,0.073517,0.067493,0.060263,0.051828,0.039635,0.029100
3,AFR,C,0.883759,0.658443,0.589312,0.518192,0.439075,0.342809,0.318503,0.290510,0.255814,0.203700,0.062396
4,EAS,UR,0.066943,0.052544,0.039876,0.039876,0.039876,0.039876,0.039876,0.039876,0.039876,0.039876,0.039876
5,EAS,R,0.127111,0.090528,0.075896,0.068213,0.063492,0.057445,0.056731,0.055395,0.055234,0.055234,0.055234
6,EAS,LF,0.535281,0.426343,0.402845,0.377764,0.347406,0.299790,0.284235,0.266284,0.236937,0.199654,0.178900
7,EAS,C,0.952998,0.874674,0.842438,0.804609,0.754409,0.669963,0.642081,0.605239,0.551307,0.453110,0.195640
8,EUR,UR,0.038198,0.021296,0.014422,0.012097,0.010686,0.009206,0.008936,0.008748,0.008572,0.008533,0.008533
9,EUR,R,0.195687,0.136144,0.121917,0.106953,0.089961,0.068111,0.062337,0.055090,0.047223,0.035091,0.023773


In [32]:
proportion_TopLD_variants_in_LD_with_AGVs_melted = pd.melt(proportion_TopLD_variants_in_LD_with_AGVs, id_vars=['ancestry_group','MAF_class'])
proportion_TopLD_variants_in_LD_with_AGVs_melted = proportion_TopLD_variants_in_LD_with_AGVs_melted.rename(columns={'variable': 'R2', 'value': 'proportion'})
proportion_TopLD_variants_in_LD_with_AGVs_melted['R2'] = proportion_TopLD_variants_in_LD_with_AGVs_melted['R2'].str.replace('R2_','',regex=True)
proportion_TopLD_variants_in_LD_with_AGVs_melted.head(5)

,ancestry_group,MAF_class,R2,proportion
0,AFR,UR,0.2,0.035293
1,AFR,R,0.2,0.107208
2,AFR,LF,0.2,0.436768
3,AFR,C,0.2,0.883759
4,EAS,UR,0.2,0.066943


In [33]:
proportion_TopLD_variants_in_LD_with_AGVs_melted.to_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_LD_variants/proportion_TopLD_variants_in_LD_with_AGVs.txt', sep='\t', header=True, index=False)

## AGV and LD Variants GWAS Intersection <a class = 'anchor' id = 'gwasintersection'></a>

Let's assess the intersection of AGV and AGV-LD variants with significant GWAS loci from ten common GWAS traits.

In [34]:
AGV_LD_variant_GWAS_intersect = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/GWAS/AGV_LD_variants_GWAS_intersection_summary.txt', sep='\t', header=0)
AGV_LD_variant_GWAS_intersect['proportion_AGV_GWAS_significant_variants'] = AGV_LD_variant_GWAS_intersect['N_AGV_GWAS_significant_variants']/AGV_LD_variant_GWAS_intersect['N_GWAS_significant_variants']
AGV_LD_variant_GWAS_intersect['proportion_AGV_and_LD_variant_GWAS_significant_variants'] = AGV_LD_variant_GWAS_intersect['N_AGV_and_LD_variant_GWAS_significant_variants']/AGV_LD_variant_GWAS_intersect['N_GWAS_significant_variants']
AGV_LD_variant_GWAS_intersect['enrichment'] = AGV_LD_variant_GWAS_intersect['N_AGV_and_LD_variant_GWAS_significant_variants']/AGV_LD_variant_GWAS_intersect['N_AGV_GWAS_significant_variants']
AGV_LD_variant_GWAS_intersect

,Trait,N_GWAS_significant_variants,N_AGV_GWAS_significant_variants,N_AGV_and_LD_variant_GWAS_significant_variants,proportion_AGV_GWAS_significant_variants,proportion_AGV_and_LD_variant_GWAS_significant_variants,enrichment
0,coronary_artery_disease,17871,2536,15033,0.141906,0.841195,5.927839
1,psoriasis,46121,5983,29133,0.129724,0.631665,4.869296
2,ulcerative_colitis,7392,1250,6078,0.169102,0.822240,4.862400
3,body_mass_index,65489,12669,57693,0.193452,0.880957,4.553872
4,N_children_ever_born,5704,274,4560,0.048036,0.799439,16.642336
5,type_2_diabetes,64918,10079,54347,0.155257,0.837164,5.392102
6,schizophrenia,20447,3152,18344,0.154155,0.897149,5.819797
7,platelet_count,145686,16736,99269,0.114877,0.681390,5.931465
8,Alzheimers,3559,424,2394,0.119135,0.672661,5.646226
9,height,115855,57558,104510,0.496811,0.902076,1.815734


In [35]:
AGV_LD_variant_GWAS_intersect['enrichment'].median()

np.float64(5.519164403102284)

In [36]:
AGV_LD_variant_GWAS_intersect.to_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/GWAS/AGV_LD_variants_GWAS_intersection_summary_updated.txt', sep='\t', header=True, index=False)

## Evaluation <a class = 'anchor' id = 'evaluation'></a>

### Mismatch Proportions by LD Threshold <a class = 'anchor' id = 'mismatchesbythreshold'></a>

Let's describe the evaluation data and analyze the results. First, how many evaluation European AGV-LDV pairs will we consider ($\textit{R}^\textit{2}$ $\geq$ 0.5)?

In [37]:
!zcat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/EUR_AGV_LD_variants/chr*_R2_filtered_EUR_AGV_LD_variants.txt.gz | wc -l

61485131


How many high-quality genotypes did we pull for Loschbour and Ust'-Ishim? Let's note the Ns after running once to speed up the notebook.

In [38]:
#!zcat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/ancient_genotypes/Loschbour_chr*_hg38.txt.gz | wc -l

1652392042

In [39]:
#!zcat /wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/ancient_genotypes/Ust_Ishim_chr*_hg38.txt.gz | wc -l

1769322914

Read in the eval stats files for Loschbour and Ust'-Ishim. Let's calculate the proportion correct for partial and complete matches at each LD threshold and output these files for visualization and inclusion in the supplement.

In [40]:
Loschbour_eval_stats = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/Loschbour_evaluation_statistics.txt', sep='\t', names = ['LD_threshold','chr','N_partial_matches','N_complete_matches','N_evaluated'])
Loschbour_eval_stats = Loschbour_eval_stats.groupby(['LD_threshold']).sum().reset_index()
Loschbour_eval_stats['partial_matches_prop'] = Loschbour_eval_stats['N_partial_matches']/Loschbour_eval_stats['N_evaluated']
Loschbour_eval_stats['complete_matches_prop'] = Loschbour_eval_stats['N_complete_matches']/Loschbour_eval_stats['N_evaluated']
Loschbour_eval_stats

,LD_threshold,chr,N_partial_matches,N_complete_matches,N_evaluated,partial_matches_prop,complete_matches_prop
0,0.5,chr1chr2chr9chr13chr10chr11chr12chr7chr4chr3ch...,28480984,27150193,28681135,0.993022,0.946622
1,0.6,chr1chr2chr9chr13chr10chr11chr12chr7chr8chr4ch...,22365711,21636605,22456534,0.995956,0.963488
2,0.7,chr1chr2chr9chr13chr10chr11chr12chr7chr8chr4ch...,17836090,17466847,17872498,0.997963,0.977303
3,0.8,chr1chr9chr13chr10chr2chr11chr12chr7chr3chr4ch...,13982723,13819534,13994787,0.999138,0.987477
4,0.9,chr1chr9chr13chr10chr2chr11chr12chr4chr3chr7ch...,10148529,10098515,10151464,0.999711,0.994784


In [41]:
Ust_Ishim_eval_stats = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/Ust_Ishim_evaluation_statistics.txt', sep='\t', names = ['LD_threshold','chr','N_partial_matches','N_complete_matches','N_evaluated'])
Ust_Ishim_eval_stats = Ust_Ishim_eval_stats.groupby(['LD_threshold']).sum().reset_index()
Ust_Ishim_eval_stats['partial_matches_prop'] = Ust_Ishim_eval_stats['N_partial_matches']/Ust_Ishim_eval_stats['N_evaluated']
Ust_Ishim_eval_stats['complete_matches_prop'] = Ust_Ishim_eval_stats['N_complete_matches']/Ust_Ishim_eval_stats['N_evaluated']
Ust_Ishim_eval_stats

,LD_threshold,chr,N_partial_matches,N_complete_matches,N_evaluated,partial_matches_prop,complete_matches_prop
0,0.5,chr9chr7chr8chr5chr1chr4chr6chr3chr2chr10chr11...,31782846,28972187,32168071,0.988025,0.900650
1,0.6,chr9chr7chr8chr6chr1chr5chr4chr3chr2chr10chr11...,24957983,23220242,25166294,0.991723,0.922672
2,0.7,chr9chr7chr8chr1chr6chr5chr4chr3chr2chr10chr11...,19914922,18889511,20017786,0.994861,0.943636
3,0.8,chr9chr7chr8chr1chr6chr5chr4chr3chr2chr10chr15...,15620562,15061695,15662043,0.997351,0.961669
4,0.9,chr9chr7chr8chr6chr1chr5chr4chr3chr2chr15chr14...,11329693,11096033,11342528,0.998868,0.978268


Let's combine the above and save as a supplemental table.

In [42]:
Loschbour_eval_stats['sample'] = 'Loschbour'
Ust_Ishim_eval_stats['sample'] = 'Ust\'-Ishim'
combined_eval_stats = pd.concat([Loschbour_eval_stats, Ust_Ishim_eval_stats], ignore_index=True)
combined_eval_stats = combined_eval_stats[['sample','LD_threshold','N_partial_matches','N_complete_matches','N_evaluated','partial_matches_prop','complete_matches_prop']]
combined_eval_stats

,sample,LD_threshold,N_partial_matches,N_complete_matches,N_evaluated,partial_matches_prop,complete_matches_prop
0,Loschbour,0.5,28480984,27150193,28681135,0.993022,0.946622
1,Loschbour,0.6,22365711,21636605,22456534,0.995956,0.963488
2,Loschbour,0.7,17836090,17466847,17872498,0.997963,0.977303
3,Loschbour,0.8,13982723,13819534,13994787,0.999138,0.987477
4,Loschbour,0.9,10148529,10098515,10151464,0.999711,0.994784
5,Ust'-Ishim,0.5,31782846,28972187,32168071,0.988025,0.900650
6,Ust'-Ishim,0.6,24957983,23220242,25166294,0.991723,0.922672
7,Ust'-Ishim,0.7,19914922,18889511,20017786,0.994861,0.943636
8,Ust'-Ishim,0.8,15620562,15061695,15662043,0.997351,0.961669
9,Ust'-Ishim,0.9,11329693,11096033,11342528,0.998868,0.978268


In [43]:
combined_eval_stats.to_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/supplemental_dfs/evaluation_stats.txt', sep='\t', header=True, index=False)

### Mismatches by Genomic Window <a class = 'anchor' id = 'mismatchesbywindow'></a>

Let's examine how many 50 kb windows had any complete mismatches.

In [44]:
Loschbour_mismatches_by_genomic_window = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/mismatches_by_genomic_window/Loschbour_0.9_complete_mismatches_by_genomic_window.txt', sep='\t', names = ['chr','pos','N_mismatched_AGVs','N_complete_mismatches','N_evaluated'])
Loschbour_mismatches_by_genomic_window['mismatch_proportion'] = Loschbour_mismatches_by_genomic_window['N_complete_mismatches']/Loschbour_mismatches_by_genomic_window['N_evaluated']
Loschbour_mismatches_by_genomic_window.head()

,chr,pos,N_mismatched_AGVs,N_complete_mismatches,N_evaluated,mismatch_proportion
0,chr1,1,0,0,0,NaN
1,chr1,25001,0,0,0,NaN
2,chr1,50001,0,0,0,NaN
3,chr1,75001,0,0,0,NaN
4,chr1,100001,0,0,0,NaN


In [45]:
len(Loschbour_mismatches_by_genomic_window)

121133

In [46]:
Loschbour_mismatches_by_genomic_window['mismatch_proportion'].mean(skipna=True)

np.float64(0.0008808455470998265)

In [47]:
Ust_Ishim_mismatches_by_genomic_window = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/evaluation/mismatches_by_genomic_window/Ust_Ishim_0.9_complete_mismatches_by_genomic_window.txt', sep='\t', names = ['chr','pos','N_mismatched_AGVs','N_complete_mismatches','N_evaluated'])
Ust_Ishim_mismatches_by_genomic_window['mismatch_proportion'] = Ust_Ishim_mismatches_by_genomic_window['N_complete_mismatches']/Ust_Ishim_mismatches_by_genomic_window['N_evaluated']
Ust_Ishim_mismatches_by_genomic_window.head()

,chr,pos,N_mismatched_AGVs,N_complete_mismatches,N_evaluated,mismatch_proportion
0,chr1,1,0,0,0,NaN
1,chr1,25001,0,0,0,NaN
2,chr1,50001,0,0,0,NaN
3,chr1,75001,0,0,0,NaN
4,chr1,100001,0,0,0,NaN


In [48]:
len(Ust_Ishim_mismatches_by_genomic_window)

121130

In [49]:
Ust_Ishim_mismatches_by_genomic_window['mismatch_proportion'].mean(skipna=True)

np.float64(0.0018867309748931567)

Is the proportion of mismatches associated with the number of mismatched AGVs for non-zero windows?

In [50]:
df = Loschbour_mismatches_by_genomic_window[
    (Loschbour_mismatches_by_genomic_window['N_mismatched_AGVs'] != 0) & 
    (~Loschbour_mismatches_by_genomic_window['N_mismatched_AGVs'].isna()) &
    (~Loschbour_mismatches_by_genomic_window['mismatch_proportion'].isna())
]
rho, p = spearmanr(df['N_mismatched_AGVs'], df['mismatch_proportion'])
rho, p

(np.float64(0.3885673333332847), np.float64(4.5427907528187e-41))

In [51]:
df = Ust_Ishim_mismatches_by_genomic_window[
    (Ust_Ishim_mismatches_by_genomic_window['N_mismatched_AGVs'] != 0) & 
    (~Ust_Ishim_mismatches_by_genomic_window['N_mismatched_AGVs'].isna()) &
    (~Ust_Ishim_mismatches_by_genomic_window['mismatch_proportion'].isna())
]
rho, p = spearmanr(df['N_mismatched_AGVs'], df['mismatch_proportion'])
rho, p

(np.float64(0.5708669210587812), np.float64(1.8800164330928658e-222))

Let's save a BED file of windows where there were mismatches.

In [52]:
Loschbour_mismatches_by_genomic_window_subset = Loschbour_mismatches_by_genomic_window[['chr', 'pos', 'mismatch_proportion']].rename(columns={'mismatch_proportion': 'Loschbour_mismatch'})
Ust_Ishim_mismatches_by_genomic_window_subset = Ust_Ishim_mismatches_by_genomic_window[['chr', 'pos', 'mismatch_proportion']].rename(columns={'mismatch_proportion': 'UstIshim_mismatch'})

merged_mismatches_by_genomic_window = pd.merge(Loschbour_mismatches_by_genomic_window_subset, Ust_Ishim_mismatches_by_genomic_window_subset, on=['chr', 'pos'], how='outer')

# Step 3: Create a label column for individuals with non-zero mismatch proportion
def identify_samples(row):
    labels = []
    if pd.notna(row['Loschbour_mismatch']) and row['Loschbour_mismatch'] > 0:
        labels.append('Loschbour')
    if pd.notna(row['UstIshim_mismatch']) and row['UstIshim_mismatch'] > 0:
        labels.append('Ust-Ishim')
    if labels:
        return ' and '.join(labels)
    else:
        return 'None'

merged_mismatches_by_genomic_window['samples'] = merged_mismatches_by_genomic_window.apply(identify_samples, axis=1)
merged_mismatches_by_genomic_window = merged_mismatches_by_genomic_window[merged_mismatches_by_genomic_window['samples'] != 'None'].copy()

merged_mismatches_by_genomic_window['start'] = merged_mismatches_by_genomic_window['pos'] - 1
merged_mismatches_by_genomic_window['end'] = merged_mismatches_by_genomic_window['pos'] + 1000000

mismatches_by_genomic_window_BED = merged_mismatches_by_genomic_window[['chr', 'start', 'end', 'samples']]

def merge_windows(df):
    df = df.sort_values(['chr', 'start'])
    merged_intervals = []
    current_chr = None
    current_samples = None
    current_start = None
    current_end = None

    for _, row in df.iterrows():
        if (row['chr'] != current_chr) or (row['samples'] != current_samples) or (current_end is None) or (row['start'] > current_end):
            # Save the previous interval if exists
            if current_chr is not None:
                merged_intervals.append([current_chr, current_start, current_end, current_samples])
            # Start a new interval
            current_chr = row['chr']
            current_samples = row['samples']
            current_start = row['start']
            current_end = row['end']
        else:
            # Overlapping or contiguous interval; extend current_end if needed
            current_end = max(current_end, row['end'])

    # Add last interval
    if current_chr is not None:
        merged_intervals.append([current_chr, current_start, current_end, current_samples])

    merged_df = pd.DataFrame(merged_intervals, columns=['chr', 'start', 'end', 'samples'])
    return merged_df

merged_mismatches_by_genomic_window_BED = merge_windows(mismatches_by_genomic_window_BED)

merged_mismatches_by_genomic_window_BED.head(5)

,chr,start,end,samples
0,chr1,2275000,3300001,Ust-Ishim
1,chr1,4825000,5850001,Ust-Ishim
2,chr1,8150000,9225001,Ust-Ishim
3,chr1,10175000,11275001,Ust-Ishim
4,chr1,13975000,15000001,Ust-Ishim


## AGVs in TopLD <a class = 'anchor' id = 'agvsintopld'></a>

Now let's characterize the AGVs themselves.

In [53]:
AGVs = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg38_annotated.txt.gz', sep = '\t', header = 0, compression = 'gzip')
AGVs.head(5)

/scratch/cbrand/ipykernel_3155226/4073827720.py:1: DtypeWarning: Columns (6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  AGVs = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGVs/AGVs_hg38_annotated.txt.gz', sep = '\t', header = 0, compression = 'gzip')


,chr,pos,ref,alt,rsID,filter,AF,AFR_AF,EAS_AF,EUR_AF,SAS_AF,TopLD_presence,MAFs,LDV_presence
0,chr1,817186,G,A,rs3094315,PASS,0.711754,0.43591,0.888097,0.825775,0.791286,NaN,NaN,NaN
1,chr1,841166,A,G,rs12124819,PASS,0.172173,0.045892,0.001348,0.272795,0.082884,"AFR,EAS,EUR,SAS","0.013,0.001,0.25,0.071","AFR,EAS,EUR,SAS"
2,chr1,897538,T,C,rs28765502,PASS,0.367279,0.548491,0.16017,0.292576,0.232684,"AFR,EAS,EUR,SAS","0.398,0.145,0.292,0.222","AFR,EAS,EUR,SAS"
3,chr1,906633,T,G,rs7419119,PASS,0.19111,0.139017,0.132745,0.208938,0.214076,"AFR,EAS,EUR,SAS","0.126,0.13,0.217,0.197","AFR,EAS,EUR,SAS"
4,chr1,911484,G,C,rs950122,PASS,0.217983,0.27126,0.103475,0.192926,0.248652,"AFR,EAS,EUR,SAS","0.274,0.107,0.195,0.274","AFR,EAS,EUR,SAS"


In [54]:
N_AGVs = len(AGVs)
N_AGVs

1232486

How many are absent from and present in TopLD?

In [55]:
N_AGVs_TopLD_absent = AGVs['TopLD_presence'].isna().sum()
N_AGVs_TopLD_absent

np.int64(69890)

In [56]:
N_AGVs_TopLD_absent / N_AGVs

np.float64(0.05670652648387081)

In [57]:
N_AGVs_TopLD_present = N_AGVs - N_AGVs_TopLD_absent
N_AGVs_TopLD_present

np.int64(1162596)

In [58]:
N_AGVs_TopLD_present / N_AGVs

np.float64(0.9432934735161292)

What are the allele frequencies of these two categories? Are they significantly different?

In [59]:
AGVs_TopLD_absent_AFs = AGVs[AGVs['TopLD_presence'].isna() & AGVs['AF'].notna()][['AF']]
AGVs_TopLD_absent_AFs= AGVs_TopLD_absent_AFs[AGVs_TopLD_absent_AFs['AF'] != '.']
AGVs_TopLD_absent_AFs['AF'] = pd.to_numeric(AGVs_TopLD_absent_AFs['AF'], errors='coerce')
AGVs_TopLD_absent_AFs.head(5)

,AF
0,0.711754
7,0.876354
48,0.000033
77,0.000046
112,0.000190


In [60]:
len(AGVs_TopLD_absent_AFs)

21794

In [61]:
AGVs_TopLD_present_AFs = AGVs[AGVs['TopLD_presence'].notna() & AGVs['AF'].notna()][['AF']]
AGVs_TopLD_present_AFs= AGVs_TopLD_present_AFs[AGVs_TopLD_present_AFs['AF'] != '.']
AGVs_TopLD_present_AFs['AF'] = pd.to_numeric(AGVs_TopLD_present_AFs['AF'], errors='coerce')
AGVs_TopLD_present_AFs.head(5)

,AF
1,0.172173
2,0.367279
3,0.191110
4,0.217983
5,0.074330


In [62]:
len(AGVs_TopLD_present_AFs)

1162237

In [63]:
AGVs_TopLD_absent_AFs['AF'].median()

np.float64(3.93786e-05)

In [64]:
AGVs_TopLD_present_AFs['AF'].median()

np.float64(0.237322)

In [65]:
mannwhitneyu(AGVs_TopLD_absent_AFs['AF'], AGVs_TopLD_present_AFs['AF'])

MannwhitneyuResult(statistic=np.float64(1983228367.5), pvalue=np.float64(0.0))

What about the distribution of AGVs among different ancestry groups?

In [66]:
AGVs.groupby('TopLD_presence').size()

TopLD_presence
AFR                 25725
AFR,EAS              1125
AFR,EAS,EUR         10260
AFR,EAS,EUR,SAS    944223
AFR,EAS,SAS         11392
AFR,EUR            110863
AFR,EUR,SAS         44651
AFR,SAS               766
EAS                  3175
EAS,EUR              1442
EAS,EUR,SAS          1874
EAS,SAS               811
EUR                  4795
EUR,SAS               891
SAS                   603
dtype: int64

Let's also take a quick peek at the gnomAD filtering.

In [67]:
AGVs.groupby('filter').size().to_frame('N')

,N
filter,
AC0,67
AC0;AS_VQSR,107
AS_VQSR,567
AS_VQSR;InbreedingCoeff,9
InbreedingCoeff,8952
PASS,1174353


In [68]:
67+107+567+9+8952

9702

In [69]:
(67+107+567+9+8952)/N_AGVs

0.00787189469089304

## AGV Age Estimates <a class = 'anchor' id = 'agvageestimates'></a>

Let's look at the estimated ages of AGVs present in the Human Genome Dating project.

In [70]:
AGV_age_estimates = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_age_estimates/AGVs_with_age_estimates_and_AFs.txt.gz', sep = '\t', header = 0, index_col = None, compression = 'gzip')
AGV_age_estimates.head(5)

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF
0,rs3094315,1,752566,G,A,.,TGP,500,499,16898.80,16901.90,16881.00,15435.50,18423.00,0.810,35562.10,35560.90,35534.60,33318.00,37869.20,0.756,38695.60,38693.70,38660.20,35801.40,41671.00,0.756,0.706299
1,rs12124819,1,776546,A,G,.,TGP,500,500,3174.73,3175.03,3172.09,2955.65,3404.37,0.920,3719.57,3719.47,3716.17,3447.67,4001.91,0.900,3076.93,3076.79,3074.46,2873.75,3286.54,0.892,0.189643
2,rs28765502,1,832918,T,C,.,TGP,500,500,20220.90,20221.90,20200.80,18613.90,21879.20,0.902,22108.40,22107.50,22089.40,20533.80,23742.50,0.902,25899.00,25908.10,25874.60,23905.60,27979.50,0.902,0.372510
3,rs7419119,1,842013,T,G,T,TGP,500,493,15138.10,15159.60,15138.10,13728.00,16693.00,0.773,19752.30,19759.40,19726.60,17682.10,21921.70,0.773,24929.90,24912.10,24859.60,22045.40,27875.00,0.773,0.189768
4,rs950122,1,846864,G,C,G,Combined,600,598,22844.60,22866.00,22844.60,20741.40,25075.50,0.478,36578.70,36594.70,36547.30,34031.90,39214.90,0.478,42263.00,42278.60,42226.70,39253.00,45386.80,0.478,0.206207


How many age estimates are there total?

In [71]:
len(AGV_age_estimates)

3126750

How many unique AGVs are there?

In [72]:
len(AGV_age_estimates['VariantID'].unique())

1117110

What is the distribution of data sources among unique variants?

In [73]:
AGV_age_estimates_grouped = AGV_age_estimates.groupby('VariantID')['DataSource'].apply(lambda x: sorted(x.unique())).reset_index()
AGV_age_estimates_grouped['DataSources'] = AGV_age_estimates_grouped['DataSource'].apply(lambda x: ' & '.join(x))
AGV_age_estimates = pd.merge(AGV_age_estimates, AGV_age_estimates_grouped[['VariantID', 'DataSources']], on = 'VariantID')
AGV_age_estimates.head()

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
0,rs3094315,1,752566,G,A,.,TGP,500,499,16898.80,16901.90,16881.00,15435.50,18423.00,0.810,35562.10,35560.90,35534.60,33318.00,37869.20,0.756,38695.60,38693.70,38660.20,35801.40,41671.00,0.756,0.706299,TGP
1,rs12124819,1,776546,A,G,.,TGP,500,500,3174.73,3175.03,3172.09,2955.65,3404.37,0.920,3719.57,3719.47,3716.17,3447.67,4001.91,0.900,3076.93,3076.79,3074.46,2873.75,3286.54,0.892,0.189643,TGP
2,rs28765502,1,832918,T,C,.,TGP,500,500,20220.90,20221.90,20200.80,18613.90,21879.20,0.902,22108.40,22107.50,22089.40,20533.80,23742.50,0.902,25899.00,25908.10,25874.60,23905.60,27979.50,0.902,0.372510,TGP
3,rs7419119,1,842013,T,G,T,TGP,500,493,15138.10,15159.60,15138.10,13728.00,16693.00,0.773,19752.30,19759.40,19726.60,17682.10,21921.70,0.773,24929.90,24912.10,24859.60,22045.40,27875.00,0.773,0.189768,TGP
4,rs950122,1,846864,G,C,G,Combined,600,598,22844.60,22866.00,22844.60,20741.40,25075.50,0.478,36578.70,36594.70,36547.30,34031.90,39214.90,0.478,42263.00,42278.60,42226.70,39253.00,45386.80,0.478,0.206207,Combined & SGDP & TGP


In [74]:
AGV_age_estimates.drop_duplicates(subset = 'VariantID').groupby('DataSources').size().to_frame('N')

,N
DataSources,
Combined & SGDP & TGP,1004811
Combined & TGP,2
SGDP,22750
SGDP & TGP,16
TGP,89531


As expected, most variants have age estimates across all three data sources. Let take a quick peak at the 18 unexpected (Combined & TGP and SGDP & TGP).

In [75]:
AGV_age_estimates_grouped[AGV_age_estimates_grouped['DataSources'] == 'SGDP & TGP']

,VariantID,DataSource,DataSources
200776,rs12111549,"[SGDP, TGP]",SGDP & TGP
217224,rs12352784,"[SGDP, TGP]",SGDP & TGP
239518,rs12624589,"[SGDP, TGP]",SGDP & TGP
257107,rs12938057,"[SGDP, TGP]",SGDP & TGP
266365,rs13113099,"[SGDP, TGP]",SGDP & TGP
326281,rs148548299,"[SGDP, TGP]",SGDP & TGP
551410,rs2813487,"[SGDP, TGP]",SGDP & TGP
558529,rs2843160,"[SGDP, TGP]",SGDP & TGP
641457,rs425535,"[SGDP, TGP]",SGDP & TGP
646109,rs4324999,"[SGDP, TGP]",SGDP & TGP


In [76]:
AGV_age_estimates[AGV_age_estimates['VariantID'] == 'rs12111549']

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
1202866,rs12111549,6,40631704,C,A,A,SGDP,99,96,21873.1,21931.4,21832.6,18749.0,25423.2,0.875,34200.2,34226.3,34133.9,28944.9,39864.7,0.875,38814.8,38822.4,38658.1,32484.6,45541.8,0.875,0.107841,SGDP & TGP
1202867,rs12111549,6,40631704,C,A,A,TGP,500,499,24887.8,24903.6,24865.8,23142.2,26741.4,0.537,39534.6,39540.8,39495.2,36392.7,42776.8,0.535,44875.2,44872.2,44830.5,41185.4,48652.4,0.535,0.107841,SGDP & TGP


In [77]:
AGV_age_estimates_grouped[AGV_age_estimates_grouped['DataSources'] == 'Combined & TGP']

,VariantID,DataSource,DataSources
116384,rs111928103,"[Combined, TGP]",Combined & TGP
133787,rs113573293,"[Combined, TGP]",Combined & TGP


In [78]:
AGV_age_estimates[AGV_age_estimates['VariantID'] == 'rs111928103']

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
2607317,rs111928103,15,88747448,G,T,G,Combined,7,587,4426.93,4412.65,4390.65,3458.29,5438.40,1.0,232.682,230.840,224.169,123.469,363.937,0.571,1222.52,1269.17,1228.02,726.229,2003.27,0.571,0.000414,Combined & TGP
2607318,rs111928103,15,88747448,G,T,G,TGP,6,494,4720.79,4710.96,4677.42,3657.01,5855.11,1.0,241.995,239.848,232.882,114.924,401.652,0.500,2838.21,2819.17,2815.18,2296.360,3340.54,1.000,0.000414,Combined & TGP


In [79]:
AGV_age_estimates[AGV_age_estimates['VariantID'] == 'rs113573293']

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
947032,rs113573293,5,7706068,G,A,G,Combined,24,584,601.94,599.402,596.956,475.605,730.812,0.750,18.441,16.698,16.226,5.057,30.62,0.042,382.383,380.091,379.259,317.935,443.227,0.792,0.000956,Combined & TGP
947033,rs113573293,5,7706068,G,A,G,TGP,21,489,601.94,600.029,596.956,476.925,732.840,0.857,18.441,16.699,16.226,5.057,30.62,0.048,389.787,387.790,387.428,324.900,450.909,0.952,0.000956,Combined & TGP


Now let's compare the age estimates across data sources for high quality variants with estimates from all three data sources. First, subset high quality estimates.

In [80]:
high_quality_AGV_age_estimates = AGV_age_estimates[AGV_age_estimates['QualScore_Jnt'] >= 0.5]
high_quality_AGV_age_estimates.head()

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
0,rs3094315,1,752566,G,A,.,TGP,500,499,16898.80,16901.90,16881.00,15435.50,18423.00,0.810,35562.10,35560.90,35534.60,33318.00,37869.20,0.756,38695.60,38693.70,38660.20,35801.40,41671.00,0.756,0.706299,TGP
1,rs12124819,1,776546,A,G,.,TGP,500,500,3174.73,3175.03,3172.09,2955.65,3404.37,0.920,3719.57,3719.47,3716.17,3447.67,4001.91,0.900,3076.93,3076.79,3074.46,2873.75,3286.54,0.892,0.189643,TGP
2,rs28765502,1,832918,T,C,.,TGP,500,500,20220.90,20221.90,20200.80,18613.90,21879.20,0.902,22108.40,22107.50,22089.40,20533.80,23742.50,0.902,25899.00,25908.10,25874.60,23905.60,27979.50,0.902,0.372510,TGP
3,rs7419119,1,842013,T,G,T,TGP,500,493,15138.10,15159.60,15138.10,13728.00,16693.00,0.773,19752.30,19759.40,19726.60,17682.10,21921.70,0.773,24929.90,24912.10,24859.60,22045.40,27875.00,0.773,0.189768,TGP
5,rs950122,1,846864,G,C,G,SGDP,100,100,18216.30,18234.10,18130.70,14875.70,21942.40,0.980,21629.60,21638.30,21519.60,17504.10,26187.70,0.980,27726.30,27725.10,27591.40,22370.90,33617.60,0.980,0.206207,Combined & SGDP & TGP


In [81]:
high_quality_AGV_age_estimates[high_quality_AGV_age_estimates['VariantID'] == 'rs182549']

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources


In [82]:
len(high_quality_AGV_age_estimates)

2294708

In [83]:
len(high_quality_AGV_age_estimates['VariantID'].unique())

954794

Let's take a look at the distribution of data sources for these variants. We will drop the old data sources column and create a new one.

In [84]:
high_quality_AGV_age_estimates = high_quality_AGV_age_estimates.drop('DataSources', axis = 1)
high_quality_AGV_age_estimates_grouped = high_quality_AGV_age_estimates.groupby('VariantID')['DataSource'].apply(lambda x: sorted(x.unique())).reset_index()
high_quality_AGV_age_estimates_grouped['DataSources'] = high_quality_AGV_age_estimates_grouped['DataSource'].apply(lambda x: ' & '.join(x))
high_quality_AGV_age_estimates = pd.merge(high_quality_AGV_age_estimates, high_quality_AGV_age_estimates_grouped[['VariantID', 'DataSources']], on = 'VariantID')
high_quality_AGV_age_estimates.head()

,VariantID,Chromosome,Position,AlleleRef,AlleleAlt,AlleleAnc,DataSource,NumConcordant,NumDiscordant,AgeMode_Mut,AgeMean_Mut,AgeMedian_Mut,AgeCI95Lower_Mut,AgeCI95Upper_Mut,QualScore_Mut,AgeMode_Rec,AgeMean_Rec,AgeMedian_Rec,AgeCI95Lower_Rec,AgeCI95Upper_Rec,QualScore_Rec,AgeMode_Jnt,AgeMean_Jnt,AgeMedian_Jnt,AgeCI95Lower_Jnt,AgeCI95Upper_Jnt,QualScore_Jnt,gnomAD_AF,DataSources
0,rs3094315,1,752566,G,A,.,TGP,500,499,16898.80,16901.90,16881.00,15435.50,18423.00,0.810,35562.10,35560.90,35534.60,33318.00,37869.20,0.756,38695.60,38693.70,38660.20,35801.40,41671.00,0.756,0.706299,TGP
1,rs12124819,1,776546,A,G,.,TGP,500,500,3174.73,3175.03,3172.09,2955.65,3404.37,0.920,3719.57,3719.47,3716.17,3447.67,4001.91,0.900,3076.93,3076.79,3074.46,2873.75,3286.54,0.892,0.189643,TGP
2,rs28765502,1,832918,T,C,.,TGP,500,500,20220.90,20221.90,20200.80,18613.90,21879.20,0.902,22108.40,22107.50,22089.40,20533.80,23742.50,0.902,25899.00,25908.10,25874.60,23905.60,27979.50,0.902,0.372510,TGP
3,rs7419119,1,842013,T,G,T,TGP,500,493,15138.10,15159.60,15138.10,13728.00,16693.00,0.773,19752.30,19759.40,19726.60,17682.10,21921.70,0.773,24929.90,24912.10,24859.60,22045.40,27875.00,0.773,0.189768,TGP
4,rs950122,1,846864,G,C,G,SGDP,100,100,18216.30,18234.10,18130.70,14875.70,21942.40,0.980,21629.60,21638.30,21519.60,17504.10,26187.70,0.980,27726.30,27725.10,27591.40,22370.90,33617.60,0.980,0.206207,SGDP


In [85]:
high_quality_AGV_age_estimates.drop_duplicates(subset = 'VariantID').groupby('DataSources').size().to_frame('N')

,N
DataSources,
Combined,351
Combined & SGDP,39949
Combined & SGDP & TGP,637502
Combined & TGP,20330
SGDP,185931
SGDP & TGP,4631
TGP,66100


Now let's subset the variants with all three data sources.

In [86]:
high_quality_AGV_age_estimates_with_three_estimates = high_quality_AGV_age_estimates.groupby('VariantID').filter(lambda x: len(x) == 3)

In [87]:
len(high_quality_AGV_age_estimates_with_three_estimates)/3

637502.0

What are the differences in age estimate across data sources?

In [88]:
high_quality_AGV_age_estimates_with_three_estimates.groupby('DataSource')['AgeMode_Jnt'].median().to_frame('Median')

,Median
DataSource,
Combined,23192.3
SGDP,21051.2
TGP,23307.0


In [89]:
high_quality_AGV_age_estimates_with_three_estimates.groupby('DataSource')['AgeMode_Jnt'].mean().to_frame('Mean')

,Mean
DataSource,
Combined,23841.758098
SGDP,22026.792095
TGP,24045.819944


In [90]:
23841.758098*29

691410.984842

In [91]:
22026.792095*29

638776.970755

In [92]:
24045.819944*29

697328.778376

In [93]:
high_quality_AGV_age_estimates_with_three_estimates_combined_estimates = high_quality_AGV_age_estimates_with_three_estimates[high_quality_AGV_age_estimates_with_three_estimates['DataSource'] == 'Combined']['AgeMode_Jnt']
high_quality_AGV_age_estimates_with_three_estimates_SGDP_estimates = high_quality_AGV_age_estimates_with_three_estimates[high_quality_AGV_age_estimates_with_three_estimates['DataSource'] == 'SGDP']['AgeMode_Jnt']
high_quality_AGV_age_estimates_with_three_estimates_TGP_estimates = high_quality_AGV_age_estimates_with_three_estimates[high_quality_AGV_age_estimates_with_three_estimates['DataSource'] == 'TGP']['AgeMode_Jnt']

In [94]:
kruskal(high_quality_AGV_age_estimates_with_three_estimates_combined_estimates, high_quality_AGV_age_estimates_with_three_estimates_SGDP_estimates, high_quality_AGV_age_estimates_with_three_estimates_TGP_estimates)

KruskalResult(statistic=np.float64(4066.3380326820334), pvalue=np.float64(0.0))

Now, let's get a single age estimate per variant. We will default to using the Combined estimates for variants where possible and SGDP/TGP estimates for variants private to those datasets. Write a function to do the filtering. Let's save the output so that we can reload because the function will take a while to run.

In [95]:
def retrieve_one_age_estimate_per_variant(variant):    
    if 'Combined' in variant['DataSource'].values:
        return variant[variant['DataSource'] == 'Combined'].iloc[0]
    elif 'TGP' in variant['DataSource'].values and 'SGDP' in variant['DataSource'].values:
        return variant[variant['DataSource'] == 'TGP'].iloc[0]
    else:
        return variant.iloc[0]

In [96]:
#high_quality_AGV_age_estimates_single_estimate_per_variant = high_quality_AGV_age_estimates.groupby('VariantID').apply(retrieve_one_age_estimate_per_variant).reset_index(drop = True)
#high_quality_AGV_age_estimates_single_estimate_per_variant.head()

In [97]:
#high_quality_AGV_age_estimates_single_estimate_per_variant.to_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_age_estimates/high_quality_AGV_age_estimates_single_estimate_per_variant.txt.gz', sep = '\t', header = True, index = False, compression = 'gzip')

Let's read in the previously generated dataframe to speed this notebook up.

In [98]:
high_quality_AGV_age_estimates_single_estimate_per_variant = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_age_estimates/high_quality_AGV_age_estimates_single_estimate_per_variant.txt.gz', sep='\t', header = 0, compression = 'gzip')

In [99]:
len(high_quality_AGV_age_estimates_single_estimate_per_variant)

954794

In [100]:
high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'].min()*29

np.float64(625.936)

In [101]:
high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'].median()*29

np.float64(695106.8)

In [102]:
high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'].mean()*29

np.float64(706343.1755468792)

In [103]:
high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'].max()*29

np.float64(2192115.8)

How many of these variants are predicted to be young (< 60,000 years old).

In [104]:
len(high_quality_AGV_age_estimates_single_estimate_per_variant[high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'] < (60000/29)])

77686

In [105]:
77686/954794

0.0813641476590762

In [106]:
high_quality_AGV_age_estimates_single_estimate_per_variant['AgeMode_Jnt'].max()*29

np.float64(2192115.8)

Now let's consider AGV ages from Wohns et al. 2022.

In [107]:
AGV_tree_sequence_ages = pd.read_csv('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/AGV_age_estimates/AGV_tree_sequence_age_estimates.txt.gz', sep='\t', header=None, names=['chr','pos','age'], compression='gzip')
AGV_tree_sequence_ages = AGV_tree_sequence_ages.dropna()
AGV_tree_sequence_ages.head(5)

,chr,pos,age
6,chr1,955641,7700.00
8,chr1,960891,1750.00
14,chr1,1083324,1580.00
15,chr1,1086035,1580.00
16,chr1,1086278,4400.04


In [108]:
len(AGV_tree_sequence_ages)

1065214

In [109]:
AGV_tree_sequence_ages['age'].min()*29

np.float64(0.0)

In [110]:
AGV_tree_sequence_ages['age'].max()*29

np.float64(2369909.0)

In [111]:
AGV_tree_sequence_ages['age'].median()*29

np.float64(323060.0)

In [112]:
AGV_tree_sequence_ages['age'].mean()*29

np.float64(416430.88678864494)

In [113]:
len(AGV_tree_sequence_ages[AGV_tree_sequence_ages['age'] < (60000/29)])

78881

In [114]:
78881/1065214

0.07405178677711709

## Selection GWAS Loci Allele Frequency Trajectories <a class = 'anchor' id = 'gwastrajectories'></a>

Let's assess the allele frequency trajectories of top effect alleles from GWAS loci under putative directional selection. We will consider both absolute beta and p-values. We've already prepared the directional selection and GWAS loci datasets. First, let's perform the intersection.

In [115]:
Akbari_et_al_2024_selection_loci = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Akbari_et_al_2024_directional_selection_loci_hg38.bed')
Irving_Pease_et_al_2024_selection_loci = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Irving_Pease_et_al_2024_directional_selection_loci_hg38.bed')

In [116]:
BMI_GWAS_significant_loci = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Koskeridis_et_al_2022_GCST90179150_BMI_significant_loci_hg38_betas_and_AFs.bed')
height_GWAS_significant_loci = pybedtools.BedTool('/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Yengo_et_al_2022_GCST90245992_significant_loci_hg38_betas_and_AFs.bed')

In [117]:
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection = pd.DataFrame([iv.fields for iv in Akbari_et_al_2024_selection_loci.intersect(BMI_GWAS_significant_loci, wb=True)], columns = ['selection_chr','selection_start','selection_end','GWAS_chr','GWAS_start','GWAS_end','GWAS_effect_allele','GWAS_other_allele','GWAS_rsID','GWAS_beta','GWAS_effect_allele_AF','GWAS_p_value'])
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_beta'] = pd.to_numeric(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_beta'], errors='coerce')
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'] = pd.to_numeric(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'], errors='coerce')
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'] = pd.to_numeric(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'], errors='coerce')
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection.head(5)

,selection_chr,selection_start,selection_end,GWAS_chr,GWAS_start,GWAS_end,GWAS_effect_allele,GWAS_other_allele,GWAS_rsID,GWAS_beta,GWAS_effect_allele_AF,GWAS_p_value
0,chr1,155166365,155166366,chr1,155166365,155166366,A,G,rs75925257,0.020390,0.1136,3.645171e-10
1,chr1,155489363,155489364,chr1,155489363,155489364,T,C,rs61812092,0.019621,0.1941,5.537706e-14
2,chr1,155852837,155852838,chr1,155852837,155852838,A,C,rs112685832,0.022516,0.1116,6.470534e-12
3,chr2,134840057,134840058,chr2,134840057,134840058,G,T,rs10496731,-0.010265,0.3830,4.129070e-09
4,chr2,134841836,134841837,chr2,134841836,134841837,C,T,rs3739029,-0.009542,0.3841,4.568065e-08


In [118]:
Akbari_et_al_2024_selection_height_GWAS_loci_intersection = pd.DataFrame([iv.fields for iv in Akbari_et_al_2024_selection_loci.intersect(height_GWAS_significant_loci, wb=True)], columns = ['selection_chr','selection_start','selection_end','GWAS_chr','GWAS_start','GWAS_end','GWAS_effect_allele','GWAS_other_allele','GWAS_rsID','GWAS_beta','GWAS_effect_allele_AF','GWAS_p_value'])
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_beta'] = pd.to_numeric(Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_beta'], errors='coerce')
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'] = pd.to_numeric(Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'], errors='coerce')
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'] = pd.to_numeric(Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'], errors='coerce')
Akbari_et_al_2024_selection_height_GWAS_loci_intersection.head(5)

,selection_chr,selection_start,selection_end,GWAS_chr,GWAS_start,GWAS_end,GWAS_effect_allele,GWAS_other_allele,GWAS_rsID,GWAS_beta,GWAS_effect_allele_AF,GWAS_p_value
0,chr1,1055036,1055037,chr1,1055036,1055037,C,T,rs2465136,0.010319,0.296,3.749840e-18
1,chr1,1070425,1070426,chr1,1070425,1070426,T,C,rs3934834,0.008759,0.150,4.999030e-09
2,chr1,56450147,56450148,chr1,56450147,56450148,A,G,rs1889145,0.007985,0.776,1.308560e-10
3,chr1,151872173,151872174,chr1,151872173,151872174,A,G,rs6684312,0.007609,0.287,5.896260e-11
4,chr1,151934696,151934697,chr1,151934696,151934697,G,A,rs10494270,0.007365,0.286,1.753140e-10


In [119]:
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection = pd.DataFrame([iv.fields for iv in Irving_Pease_et_al_2024_selection_loci.intersect(BMI_GWAS_significant_loci, wb=True)], columns = ['selection_chr','selection_start','selection_end','GWAS_chr','GWAS_start','GWAS_end','GWAS_effect_allele','GWAS_other_allele','GWAS_rsID','GWAS_beta','GWAS_effect_allele_AF','GWAS_p_value'])
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_beta'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_beta'], errors='coerce')
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'], errors='coerce')
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'], errors='coerce')
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection.head(5)

,selection_chr,selection_start,selection_end,GWAS_chr,GWAS_start,GWAS_end,GWAS_effect_allele,GWAS_other_allele,GWAS_rsID,GWAS_beta,GWAS_effect_allele_AF,GWAS_p_value
0,chr11,27673287,27673288,chr11,27673287,27673288,G,A,rs11030107,0.027193,0.2685,3.472617e-51
1,chr11,27673916,27673917,chr11,27673916,27673917,G,A,rs11030108,-0.028736,0.6687,4.947709e-64
2,chr11,27723311,27723312,chr11,27723311,27723312,C,T,rs12273363,0.027519,0.1978,1.121069e-42
3,chr11,27880404,27880405,chr11,27880404,27880405,T,C,rs61890097,-0.020278,0.0908,1.651595e-08
4,chr11,43843390,43843391,chr11,43843390,43843391,A,C,rs117242440,0.022236,0.0902,6.762752e-10


In [120]:
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection = pd.DataFrame([iv.fields for iv in Irving_Pease_et_al_2024_selection_loci.intersect(height_GWAS_significant_loci, wb=True)], columns = ['selection_chr','selection_start','selection_end','GWAS_chr','GWAS_start','GWAS_end','GWAS_effect_allele','GWAS_other_allele','GWAS_rsID','GWAS_beta','GWAS_effect_allele_AF','GWAS_p_value'])
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_beta'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_beta'], errors='coerce')
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'], errors='coerce')
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'] = pd.to_numeric(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'], errors='coerce')
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection.head(5)

,selection_chr,selection_start,selection_end,GWAS_chr,GWAS_start,GWAS_end,GWAS_effect_allele,GWAS_other_allele,GWAS_rsID,GWAS_beta,GWAS_effect_allele_AF,GWAS_p_value
0,chr10,17114151,17114152,chr10,17114151,17114152,G,A,rs1801222,0.008188,0.643,5.072460e-14
1,chr10,24889934,24889935,chr10,24889934,24889935,A,G,rs11014285,0.022590,0.151,2.076940e-52
2,chr11,1995677,1995678,chr11,1995677,1995678,A,G,rs217727,-0.009386,0.184,1.706660e-11
3,chr11,2129213,2129214,chr11,2129213,2129214,C,T,rs2585,0.016206,0.720,3.391970e-43
4,chr11,27660048,27660049,chr11,27660048,27660049,G,C,rs11030102,0.006644,0.254,3.039200e-08


In [121]:
len(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection)

1287

In [122]:
len(Akbari_et_al_2024_selection_height_GWAS_loci_intersection)

2085

In [123]:
len(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection)

179

In [124]:
len(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection)

222

Now let's subset and save the largest effect loci per intersection.

In [125]:
def subset_largest_effect_loci(df, out_file, beta_col='GWAS_beta', top_percent=20):
    """
    Filters a dataframe for top N% of variants by absolute beta and saves results.
    
    Parameters:
        df (pd.DataFrame): Input dataframe.
        out_file (str): Filename to save top N% dataframe.
        beta_col (str): Column name containing beta values.
        top_percent (float): Percentage of top absolute betas to keep.
    
    Returns:
        largest_effect_alleles_df (pd.DataFrame): Top N% by absolute beta.
    """  
    df_filtered = df[np.isfinite(df[beta_col])].copy()
    
    threshold = np.percentile(np.abs(df_filtered[beta_col]), 100 - top_percent)
    print(f"Top {top_percent}% cutoff (absolute beta): {threshold:.6f}")
    
    largest_effect_alleles_df = df_filtered[np.abs(df_filtered[beta_col]) >= threshold].copy()
    
    largest_effect_alleles_df[['GWAS_chr','GWAS_rsID','GWAS_effect_allele','GWAS_other_allele','GWAS_beta']].to_csv(out_file, sep='\t', header=False, index=False)
    
    print(f"Saved {len(largest_effect_alleles_df)} largest effect loci.")

In [126]:
subset_largest_effect_loci(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Akbari_et_al_2024_selection_BMI_GWAS_largest_effect_loci.txt',
                              beta_col='GWAS_beta',
                              top_percent=20
)

Top 20% cutoff (absolute beta): 0.017278
Saved 258 largest effect loci.


In [127]:
subset_largest_effect_loci(Akbari_et_al_2024_selection_height_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Akbari_et_al_2024_selection_height_GWAS_largest_effect_loci.txt',
                              beta_col='GWAS_beta',
                              top_percent=20
)

Top 20% cutoff (absolute beta): 0.025022
Saved 417 largest effect loci.


In [128]:
subset_largest_effect_loci(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Irving_Pease_et_al_2024_selection_BMI_GWAS_largest_effect_loci.txt',
                              beta_col='GWAS_beta',
                              top_percent=20
)

Top 20% cutoff (absolute beta): 0.021839
Saved 36 largest effect loci.


In [129]:
subset_largest_effect_loci(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Irving_Pease_et_al_2024_selection_height_GWAS_largest_effect_loci.txt',
                              beta_col='GWAS_beta',
                              top_percent=20
)

Top 20% cutoff (absolute beta): 0.027432
Saved 45 largest effect loci.


Let's examine the effect allele frequencies and p-value distributions per intersection.

In [130]:
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'].min()

np.float64(0.0374)

In [131]:
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'].min()

np.float64(0.0101)

In [132]:
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_effect_allele_AF'].min()

np.float64(0.0221)

In [133]:
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_effect_allele_AF'].min()

np.float64(0.0141)

In [134]:
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].max()

np.float64(4.83041822304431e-08)

In [135]:
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].median()

np.float64(2.59345939965106e-13)

In [136]:
Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].min()

np.float64(3.99020545374436e-46)

In [137]:
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].max()

np.float64(4.65783e-08)

In [138]:
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].median()

np.float64(3.8417e-22)

In [139]:
Akbari_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].min()

np.float64(0.0)

In [140]:
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].max()

np.float64(4.78920494849895e-08)

In [141]:
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].median()

np.float64(1.0612984289181e-14)

In [142]:
Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection['GWAS_p_value'].min()

np.float64(4.94770898506879e-64)

In [143]:
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].max()

np.float64(3.0392e-08)

In [144]:
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].median()

np.float64(9.250845e-32)

In [145]:
Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection['GWAS_p_value'].min()

np.float64(0.0)

Now let's subset and save the strongest p-value loci per intersection.

In [146]:
def subset_strongest_p_value_loci(df, out_file, beta_col='GWAS_beta', p_value_col='GWAS_p_value', top_percent=20):
    """
    Filters a dataframe for the strongest (lowest) N% of variants by p-value and saves results.
    
    Parameters:
        df (pd.DataFrame): Input dataframe.
        out_file (str): Filename to save results.
        beta_col (str): Column name containing beta values.
        p_value_col (str): Column containing p-values.
        top_percent (float): Percentage of strongest associations (smallest p-values) to keep.
    
    Returns:
        strongest_p_value_df (pd.DataFrame): Top N% by significance.
    """
    df_filtered = df[np.isfinite(df[p_value_col])].copy()

    cutoff = np.percentile(df_filtered[p_value_col], top_percent)

    print(f"Top {top_percent}% cutoff (p-value ≤): {cutoff:.20g}")

    strongest_p_value_df = df_filtered[df_filtered[p_value_col] <= cutoff].copy()

    strongest_p_value_df[
        ['GWAS_chr','GWAS_rsID','GWAS_effect_allele','GWAS_other_allele','GWAS_beta','GWAS_p_value']
    ].to_csv(out_file, sep='\t', header=False, index=False)

    print(f"Saved {len(strongest_p_value_df)} strongest p-value loci.")

In [147]:
subset_strongest_p_value_loci(Akbari_et_al_2024_selection_BMI_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Akbari_et_al_2024_selection_BMI_GWAS_strongest_p_value_loci.txt',
                              p_value_col='GWAS_p_value',
                              top_percent=20
)

Top 20% cutoff (p-value ≤): 1.1370695547392317041e-18
Saved 258 strongest p-value loci.


In [148]:
subset_strongest_p_value_loci(Akbari_et_al_2024_selection_height_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Akbari_et_al_2024_selection_height_GWAS_strongest_p_value_loci.txt',
                              p_value_col='GWAS_p_value',
                              top_percent=20
)

Top 20% cutoff (p-value ≤): 1.2725120000000011532e-58
Saved 417 strongest p-value loci.


In [149]:
subset_strongest_p_value_loci(Irving_Pease_et_al_2024_selection_BMI_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Irving_Pease_et_al_2024_selection_BMI_GWAS_strongest_p_value_loci.txt',
                              p_value_col='GWAS_p_value',
                              top_percent=20
)

Top 20% cutoff (p-value ≤): 1.3347900878031841882e-20
Saved 36 strongest p-value loci.


In [150]:
subset_strongest_p_value_loci(Irving_Pease_et_al_2024_selection_height_GWAS_loci_intersection,
                              out_file='/wynton/group/capra/projects/ancient_genotyped_variants_proxy_catalog/data/selection_GWAS_loci_allele_frequency_trajectories/Irving_Pease_et_al_2024_selection_height_GWAS_strongest_p_value_loci.txt',
                              p_value_col='GWAS_p_value',
                              top_percent=20
)

Top 20% cutoff (p-value ≤): 4.5507896000000625896e-92
Saved 45 strongest p-value loci.
